# 🔌 MCP Workshop: Giving AI Agents Superpowers

## What is MCP (Model Context Protocol)?

MCP is an **open protocol** that enables AI models to securely access external tools and data sources. Think of it as a standardized way for AI agents to:

- 📂 **Read files** and access filesystems
- 🗄️ **Query databases** and APIs
- ⚡ **Take actions** like generating reports or sending notifications

In this workshop, you'll:
1. Start an MCP server as a subprocess
2. Connect to it via stdio transport
3. Discover and use tools programmatically
4. Build an AI agent that leverages MCP tools

---

## 🚀 Getting Started

In [1]:
# Cell 1: Install Dependencies
# This cell installs all required packages for the MCP workshop

import subprocess
import sys

print("📦 Installing required packages...")

packages = [
    "mcp",           # MCP SDK for Python
    "nest-asyncio",  # Needed for asyncio in notebooks
    "pydantic",      # Data validation
]

for pkg in packages:
    print(f"   Installing {pkg}...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

print("\n✅ All dependencies installed!")

📦 Installing required packages...
   Installing mcp...
   Installing nest-asyncio...
   Installing pydantic...

✅ All dependencies installed!


## 🖥️ Step 2: Start the MCP Server as a Subprocess

The MCP server runs as a **separate process** and communicates via **stdio** (stdin/stdout). This is the production-ready pattern for MCP:

```
┌─────────────┐    stdin/stdout    ┌─────────────┐
│  Notebook   │ ◄──────────────► │ MCP Server  │
│  (Client)   │    JSON-RPC       │ (Subprocess)│
└─────────────┘                   └─────────────┘
```

The cell below:
1. Spawns `run_mcp_server.py` as a background subprocess
2. The `StdioMCPClient` handles all the low-level communication
3. You can then call tools through the client

In [2]:
# Cell 2: Start MCP Server and Connect
# The MCP server is started as a subprocess - you don't need a separate terminal!

import nest_asyncio
nest_asyncio.apply()

import asyncio
import atexit
import signal
import sys
import os

# Add the current directory to path for imports
sys.path.insert(0, os.getcwd())

from stdio_mcp_client import StdioMCPClient, SyncMCPClient

# Create the async client
async_client = StdioMCPClient()

# Connect to the MCP server (this AUTOMATICALLY spawns the server as a subprocess!)
print("🔌 Starting MCP server subprocess and connecting...")
print("   (The server runs in the background - no separate terminal needed!)\n")

try:
    tools = await async_client.connect(
        command=sys.executable,  # Use the same Python interpreter
        args=["run_mcp_server.py"]
    )
    
    print(f"✅ Connected! Server is running as subprocess (PID managed by MCP SDK)")
    print(f"📦 Discovered {len(tools)} tools:\n")
    
    for tool in tools:
        print(f"   • {tool.name}: {tool.description}")
    
    # Create sync wrapper for easier use
    client = SyncMCPClient(async_client)
    
    # Register cleanup on notebook shutdown
    def cleanup():
        try:
            asyncio.get_event_loop().run_until_complete(async_client.close())
            print("🧹 MCP client closed")
        except:
            pass
    
    atexit.register(cleanup)
    
    print("\n🎯 Client ready! Use `client.call_tool(name, args)` to call tools")
    
except Exception as e:
    print(f"❌ Failed to connect: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Make sure run_mcp_server.py exists in the current directory")
    print("   2. Check that mcp package is installed")
    raise

🔌 Starting MCP server subprocess and connecting...
   (The server runs in the background - no separate terminal needed!)

✅ Connected! Server is running as subprocess (PID managed by MCP SDK)
📦 Discovered 8 tools:

   • read_file: Read the contents of a file. Returns the text content.
   • list_directory: List files and folders in a directory.
   • query_products: Query products database. Filter by category, price range, or search name.
   • query_sales: Query sales data by region, date range, or product.
   • get_analytics: Get analytics: revenue, top_products, sales_by_region, inventory_value.
   • generate_report: Generate and save a markdown report.
   • send_notification: Send notification to slack/email/teams (simulated).
   • create_task: Create a task/todo item.

🎯 Client ready! Use `client.call_tool(name, args)` to call tools


## 🧪 Step 3: Test the MCP Tools

Now let's verify the connection by calling some tools directly. The `client.call_tool()` method sends a request to the MCP server, which executes the tool and returns the result.

### Available Tool Categories:

| Category | Tools | Description |
|----------|-------|-------------|
| **Database** | `query_products`, `query_sales`, `get_analytics` | Query product/sales data |
| **Filesystem** | `read_file`, `list_directory` | Access files safely |
| **Actions** | `generate_report`, `send_notification`, `create_task` | Perform actions |

In [3]:
# Test 1: Query the products database
# This tool queries an in-memory SQLite database managed by the MCP server

print("🔍 Calling: query_products(category='Electronics')\n")
result = client.call_tool("query_products", {"category": "Electronics"})
print(result)

🔍 Calling: query_products(category='Electronics')

[
  {
    "id": 1,
    "name": "Widget Pro",
    "category": "Electronics",
    "price": 299.99,
    "stock": 150
  },
  {
    "id": 2,
    "name": "Gadget Plus",
    "category": "Electronics",
    "price": 199.99,
    "stock": 75
  }
]


In [4]:
# Test 2: Get analytics
# The analytics tool aggregates data and performs calculations server-side

print("📊 Calling: get_analytics(metric='revenue')\n")
result = client.call_tool("get_analytics", {"metric": "revenue"})
print(result)

📊 Calling: get_analytics(metric='revenue')

{
  "total_revenue": 23497.8,
  "total_transactions": 6,
  "total_units_sold": 220
}


In [5]:
# Test 3: List files in a directory
# The filesystem tools are sandboxed - they can only access allowed paths

print("📁 Calling: list_directory(path='files')\n")
result = client.call_tool("list_directory", {"path": "files"})
print(result)

📁 Calling: list_directory(path='files')

[
  {
    "name": "q4_report.txt",
    "type": "file"
  },
  {
    "name": "sales_forecast.txt",
    "type": "file"
  },
  {
    "name": "product_feedback.txt",
    "type": "file"
  },
  {
    "name": "incident_log.txt",
    "type": "file"
  },
  {
    "name": "customer_summary.txt",
    "type": "file"
  },
  {
    "name": "inventory_alert.txt",
    "type": "file"
  }
]


---

## 🤖 Step 4: Build an AI Agent with MCP Tools

Now for the powerful part: connecting MCP tools to an **AI agent** that can reason about which tools to use.

The agent uses a **ReAct** (Reasoning + Acting) pattern:
1. **Thought**: The LLM reasons about what to do
2. **Action**: It chooses a tool and parameters
3. **Observation**: It sees the result
4. **Repeat** until it can give a final answer

```
User Question → LLM thinks → Calls MCP Tool → Gets Result → LLM answers
```

In [ ]:
# Create an AI Agent powered by MCP tools
from mcp_agent import MCPAgent

# The agent wraps our MCP client and connects it to an LLM
agent = MCPAgent(client)

print("🤖 AI Agent created!")
print(f"🔧 Connected to MCP server with {len(client.get_tools_list())} tools:")
for tool_name in client.get_tools_list():
    print(f"   • {tool_name}")
print("\n💬 Ask the agent anything about products, sales, or files!")

🔧 Using BedrockBridge for LLM calls
🤖 AI Agent created!
🔧 Connected to MCP server with 8 tools:
   • read_file
   • list_directory
   • query_products
   • query_sales
   • get_analytics
   • generate_report
   • send_notification
   • create_task

💬 Ask the agent anything about products, sales, or files!


In [ ]:
# Example 1: The agent reasons and uses tools to answer
# Watch the Thought → Action → Observation → Answer flow!

agent.run("What Electronics products do we have and which is the most expensive?")

🤖 Agent starting...

💭 Thought: I need to query the products database to find all electronics products, then identify which one is the most expensive.

Action: query_products
Action Input: {"category": "Electronics"}
Observation: [
  {
    "id": 1,
    "name": "Widget Pro",
    "category": "Electronics",
    "price": 299.99,
    "stock": 150
  },
  {
    "id": 2,
    "name": "Gadget Plus",
    "category": "Electronics",
    "price": 199.99,
    "stock": 75
  }
]

💭 LLM Response:
Thought: I have the Electronics products from the database. Now I can see what products we have and identify which one is the most expensive by comparing their prices.

Final Answer: We have 2 Electronics products in our inventory:

1. **Widget Pro** - $299.99 (150 units in stock)
2. **Gadget Plus** - $199.99 (75 units in stock)

The most expensive Electronics product is the **Widget Pro** at $299.99.


Final Answer: We have 2 Electronics products in our inventory:

1. **Widget Pro** - $299.99 (150 units in sto

In [7]:
# Example 2: Revenue analysis
agent.run("What is our total revenue across all sales?")

🤖 Agent starting...

💭 Thought: I need to get the total revenue across all sales. I can use the get_analytics tool with the "revenue" metric to get this information.

Action: get_analytics
Action Input: {"metric": "revenue"}
Observation: {
  "total_revenue": 23497.8,
  "total_transactions": 6,
  "total_units_sold": 220
}

💭 LLM Response:
Final Answer: Our total revenue across all sales is $23,497.80. This comes from 6 total transactions with 220 units sold in total.


Final Answer: Our total revenue across all sales is $23,497.80. This comes from 6 total transactions with 220 units sold in total.


✅ Agent finished!


In [ ]:
# Example 3: Multi-step reasoning (agent may need multiple tool calls)
agent.run("Show me the top 3 products by revenue")

In [ ]:
# Example 4: File operations
agent.run("What files are in the 'files' directory?")

---

# 🏋️ Challenges

Now it's your turn! Complete the following exercises to deepen your understanding of MCP.

---

## Challenge 1: Multi-Tool Query (Easy)

Ask the agent a question that requires using **multiple tools**. For example, you might want to find sales data for a specific product category, which requires first querying products, then querying sales.

In [ ]:
# 🎯 Challenge 1: YOUR TURN
# Write a prompt that requires the agent to use at least 2 different tools
# Example ideas:
#   - "Which region has the highest sales revenue and what products were sold there?"
#   - "Find all products under $100 and tell me their total inventory value"

agent.run("YOUR_PROMPT_HERE")

---

## Challenge 2: Generate a Report (Medium)

Ask the agent to analyze some data and then **generate a report** using the `generate_report` tool. The report should be saved to the `output/` directory.

In [ ]:
# 🎯 Challenge 2: YOUR TURN
# Ask the agent to generate a sales report
# Hint: The agent should first get data, then use generate_report

agent.run("Analyze our sales by region and generate a report summarizing the findings")

---

## Challenge 3: Direct Tool Calls (Medium)

Sometimes you want to call tools **directly** without the AI agent. Use the `client.call_tool()` method to:

1. Read the contents of `files/q4_report.txt`
2. Get the inventory value analytics
3. Create a task for follow-up

In [ ]:
# 🎯 Challenge 3: YOUR TURN - Direct Tool Calls
# Fill in the arguments for each tool call

# Step 1: Read a file
report_content = client.call_tool("read_file", {
    "path": "files/q4_report.txt"  # ✅ This one is done for you
})
print("📄 Q4 Report:\n", report_content[:500], "...\n")

# Step 2: Get inventory value (fill in the arguments)
inventory = client.call_tool("get_analytics", {
    # YOUR CODE HERE - what metric should you use?
})
print("📦 Inventory Value:\n", inventory, "\n")

# Step 3: Create a task (fill in the arguments)
task = client.call_tool("create_task", {
    # YOUR CODE HERE - create a task with title, description, and priority
})
print("✅ Task Created:\n", task)

---

## Challenge 4: Add a New Tool (Hard) 🔥

Extend the MCP server with a **new tool**! 

Open `mcp_servers.py` and add a new tool called `get_low_stock_products` that returns products with stock below a threshold.

**Steps:**
1. Add the tool definition in `CombinedMCPServer._setup_handlers()` 
2. Add the handler method
3. Restart the kernel and run the notebook again
4. Test your new tool below!

In [ ]:
# 🎯 Challenge 4: Test your new tool!
# Uncomment and run after implementing get_low_stock_products in mcp_servers.py

# result = client.call_tool("get_low_stock_products", {"threshold": 100})
# print(result)

# Or use the agent:
# agent.run("Which products have low stock and need reordering?")

---

## Challenge 5: End-to-End Workflow (Hard) 🔥

Create an **automated workflow** that:
1. Reads the `files/sales_forecast.txt` file
2. Gets current sales analytics by region
3. Generates a comparison report
4. Sends a notification about the findings
5. Creates a follow-up task

You can use either the agent OR direct tool calls. Try both approaches!

In [ ]:
# 🎯 Challenge 5: YOUR TURN - End-to-End Workflow

# Option A: Use the agent (let AI figure out the steps)
# agent.run("""
#     Read the sales forecast file, compare it with actual sales by region,
#     generate a report with the comparison, send a Slack notification to #sales
#     with key findings, and create a task for the sales team to review.
# """)

# Option B: Direct tool calls (you control each step)
# Step 1: Read forecast
# forecast = client.call_tool("read_file", {"path": "files/sales_forecast.txt"})

# Step 2: Get actual sales
# actuals = client.call_tool("get_analytics", {"metric": "sales_by_region"})

# Step 3: Generate report
# report = client.call_tool("generate_report", {
#     "title": "Sales Forecast vs Actuals",
#     "content": f"...",  # combine forecast and actuals
#     "filename": "forecast_comparison"
# })

# Step 4: Send notification
# notification = client.call_tool("send_notification", {
#     "channel": "slack",
#     "recipient": "#sales",
#     "message": "New forecast comparison report available!"
# })

# Step 5: Create task
# task = client.call_tool("create_task", {
#     "title": "Review forecast comparison",
#     "assignee": "sales-team",
#     "priority": "high"
# })

print("💡 Uncomment the approach you want to try!")

---

## 🧹 Cleanup

When you're done experimenting, run this cell to cleanly close the MCP connection:

In [ ]:
# Cleanup: Close the MCP connection
# This stops the server subprocess gracefully

import asyncio

try:
    await async_client.close()
    print("✅ MCP connection closed")
    print("🛑 Server subprocess terminated")
except Exception as e:
    print(f"⚠️ Cleanup warning: {e}")

---

## 📚 What You Learned

✅ **MCP Architecture**: Server/client model with stdio transport  
✅ **Tool Discovery**: Using `list_tools()` to discover available capabilities  
✅ **Tool Execution**: Calling tools with `call_tool()` and handling responses  
✅ **AI Integration**: Connecting MCP tools to an LLM agent (ReAct pattern)  
✅ **Subprocess Model**: Running servers as isolated processes

## 🚀 Next Steps

1. **Explore the code**: Read `mcp_servers.py` to understand how tools are implemented
2. **Add more tools**: Try the Challenge 4 exercise to extend the server
3. **Connect to real services**: Replace simulated tools with real API calls
4. **Build your own agent**: Customize `mcpagentDef.py` for your use case
5. **Try MCP with other LLMs**: The protocol works with any AI model!

## 🔗 Resources

- [MCP Specification](https://modelcontextprotocol.io)
- [MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk)
- [MCP Server Examples](https://github.com/modelcontextprotocol/servers)